In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import CenteredNorm
from scipy.stats import spearmanr
import numpy as np
import pandas as pd
import deconomix
import scanpy as sc
from pathlib import Path

In [2]:
X_ref = pd.read_csv("../Data/cell_type_reference_matrix.csv", index_col=0)
adata_st = sc.read_h5ad("../Data/STDS0000002/adata_processed.h5ad")
print(adata_st)
print(adata_st.var_names[:5])

AnnData object with n_obs × n_vars = 39936 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'section'
    obsm: 'spatial'
Index(['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3'], dtype='object')


/Users/bilalkachir/Desktop/Internship/.venv/lib/python3.11/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
# Clean up index: remove tab and whitespace
X_ref.index = X_ref.index.str.split("\t").str[0].str.strip()

In [4]:
import mygene
mg = mygene.MyGeneInfo()

# Convert Ensembl IDs to gene symbols
ensembl_ids = X_ref.index.tolist()
query = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='human')
mapping = {entry['query']: entry.get('symbol', None) for entry in query if 'symbol' in entry}

# Drop any that failed to map
X_ref = X_ref.rename(index=mapping).dropna()

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
21 input query terms found dup hits:	[('ENSG00000188660', 2), ('ENSG00000215156', 2), ('ENSG00000226519', 2), ('ENSG00000227110', 2), ('E
521 input query terms found no hit:	['ENSG00000112096', 'ENSG00000131484', 'ENSG00000132832', 'ENSG00000137808', 'ENSG00000139656', 'ENS


In [5]:
print("First few genes in X_ref:", X_ref.index[:5].tolist())
print("First few genes in adata_st:", adata_st.var_names[:5].tolist())

First few genes in X_ref: ['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'FIRRM']
First few genes in adata_st: ['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3']


In [6]:
# STEP 1: Compute variance across cell types
gene_variance = X_ref.var(axis=1)

# STEP 2: Sort and get top 2000 genes
top_genes = gene_variance.sort_values(ascending=False).head(2000).index

# STEP 3: Intersect with genes in ST dataset
genes_st = set(adata_st.var_names)
genes_final = top_genes.intersection(genes_st)

# STEP 4: Reduce X_ref to selected genes
X_ref_reduced = X_ref.loc[genes_final]

# Save
X_ref_reduced.to_csv("../Data/X_reference_top2000.csv")

In [7]:
adata = sc.read_h5ad("../Data/scatlas/E-MTAB-9543_reference_data.h5ad")
# Remove duplicated Ensembl ID after tab and strip whitespace
adata.var_names = adata.var_names.str.split("\t").str[0].str.strip()
print(adata.var_names[:10])
adata.write("../Data/scatlas/E-MTAB-9543_reference_data.h5ad")

mg = mygene.MyGeneInfo()

# Query mygene service
query = mg.querymany(
    adata.var_names.tolist(),
    scopes="ensembl.gene",
    fields="symbol",
    species="human"
)

# Build a mapping: Ensembl ID → Gene Symbol
ensembl_to_symbol = {
    item["query"]: item["symbol"]
    for item in query if "symbol" in item
}

# Filter only valid symbols
valid_genes = [g for g in adata.var_names if g in ensembl_to_symbol]

# Subset and rename
adata = adata[:, valid_genes].copy()
adata.var_names = [ensembl_to_symbol[g] for g in valid_genes]


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


Index(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000000971', 'ENSG00000001036', 'ENSG00000001084',
       'ENSG00000001167'],
      dtype='object')


21 input query terms found dup hits:	[('ENSG00000188660', 2), ('ENSG00000215156', 2), ('ENSG00000226519', 2), ('ENSG00000227110', 2), ('E
521 input query terms found no hit:	['ENSG00000112096', 'ENSG00000131484', 'ENSG00000132832', 'ENSG00000137808', 'ENSG00000139656', 'ENS


In [8]:
# Ensure adata.var_names are unique (needed for column indexing)
adata.var_names_make_unique()

# Filter genes_final to only those present in adata
genes_final_valid = [g for g in genes_final if g in adata.var_names]

n_max = 10000
sc_ref_frames = []

for cell_type in adata.obs["cell_type"].unique():
    subset = adata[adata.obs["cell_type"] == cell_type]

    # Subset to top genes
    subset = subset[:, genes_final]

    # Sample cells
    if subset.n_obs > n_max:
        subset = subset[np.random.choice(subset.obs_names, n_max, replace=False)]

    # Get matrix and convert to dense if needed
    X = subset.X
    if hasattr(X, "toarray"):
        X = X.toarray()
    
    # Create DataFrame
    df = pd.DataFrame(X.T, index=genes_final)
    df.columns = [cell_type] * subset.n_obs  # Label columns by cell type

    sc_ref_frames.append(df)

# STEP 6: Combine all samples into one big DataFrame
sc_references = pd.concat(sc_ref_frames, axis=1)

# Save the DataFrame
sc_references.to_csv("../Data/sc_references_top2000.csv")

# Summary
print(f"Final reference shape: {X_ref_reduced.shape}")
print(f"sc_references shape: {sc_references.shape}")

Final reference shape: (1549, 7)
sc_references shape: (1548, 12315)
